In [1]:
import numpy as np
import random
from sklearn.metrics.pairwise import pairwise_kernels, linear_kernel, polynomial_kernel,rbf_kernel,sigmoid_kernel,laplacian_kernel
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
from pycalibration import ca
import time

In [10]:
import numpy as np

def SKCE(kernel, predictions, outcomes):
    # Reshape predictions and outcomes into 2D arrays
    predictions = np.array(predictions).reshape(-1, 1)
    outcomes = np.array(outcomes).reshape(-1, 1)

    n = len(outcomes)

    # Initialize SKCE
    skce_matrix = np.zeros((n,n))

    # For each pair of data points
    for i in range(n):
        for j in range(i+1,n):
            # Compute the difference between unit vectors outcomes and predictions
            diff_Yi_g_Xi = outcomes[i] - predictions[i]
            diff_Yj_g_Xj = outcomes[j] - predictions[j]

            # Reshape the differences to be 2D arrays
            diff_Yi_g_Xi = diff_Yi_g_Xi.reshape(1, -1)
            diff_Yj_g_Xj = diff_Yj_g_Xj.reshape(1, -1)

            # Compute the kernel matrix for the pair
            k_g_Xi_g_Xj = kernel(predictions[i].reshape(1, -1), predictions[j].reshape(1, -1))

            print(diff_Yi_g_Xi, k_g_Xi_g_Xj, diff_Yj_g_Xj)
            # Compute the hi,j term
            skce_matrix[i,j] = diff_Yi_g_Xi.T * k_g_Xi_g_Xj * diff_Yj_g_Xj

    skce = np.sum(skce_matrix)

    
    # Divide the SKCE by n over 2
    prefactor = (n* (n-1)) // 2
    skce /= prefactor

    return skce


def transform_outcomes(predictions, outcomes):
    #creates vector for outcomes to match predictions shape
    transformed_outcomes = np.zeros_like(predictions)
    transformed_outcomes[np.arange(len(predictions)), outcomes] = 1
    return transformed_outcomes

In [11]:
from sklearn.gaussian_process.kernels import RBF as RBF_GP
from sklearn.metrics.pairwise import rbf_kernel as RBF_PW

# Instantiate the RBF kernel from sklearn.gaussian_process.kernels
rbf_gp = RBF_GP(length_scale=1)

# Use the rbf_kernel function from sklearn.metrics.pairwise
def rbf_pw(X, Y=None, length_scale=1.0):
    gamma = 1.0 / (2 * length_scale ** 2)
    return RBF_PW(X, Y, gamma=gamma)

In [30]:
rng = np.random.default_rng(1234)
import random
random.seed(1234)
#predictions = np.array([random.random() for _ in range(100)])
#outcomes = np.array([bool(random.getrandbits(1)) for _ in range(100)])
#outcomes = predictions.copy()
#kommt discrepanz von Kernel vs Impelentierung

#ECE bei Ensemble heterogene / homogene
#KCE vs. ECE ??

predictions = np.array([ 0.1, 0.9])
outcomes = np.array([ 0, 1])
outcomes = np.array([False,True])



# Now you can use these in your calculate_SKCE function
SKCE(rbf_pw, predictions, outcomes)
#print(SKCE(rbf_pw, predictions, outcomes))


[[-0.1]] [[0.72614904]] [[0.1]]


-0.007261490370736909

In [31]:
skce_julia = ca.SKCE(ca.RBFKernel(), unbiased=True)
skce_julia(predictions, outcomes)

-0.005714347651353832

In [9]:
# Measure the time taken by function1
start_time = time.time()
SKCE(rbf_gp, predictions, outcomes)
end_time = time.time()
print(f"SKCE Python took {end_time - start_time} seconds to execute.")

# Measure the time taken by function2
start_time = time.time()
skce_julia(predictions, outcomes)
end_time = time.time()
print(f"SKCE Julia took {end_time - start_time} seconds to execute.")

SKCE Python took 0.23659086227416992 seconds to execute.
SKCE Julia took 0.0029761791229248047 seconds to execute.
